In [0]:
## Import libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import math
import time
from google.colab import drive

In [2]:
## IMport tensorflow
try:
  %tensorflow_version 2.x
except:
  pass

import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

TensorFlow 2.x selected.


In [3]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cols = ["sentiment","id","date","query","user","text"]
train_data = pd.read_csv(
    "/content/drive/My Drive/nlp/train.csv",
    header=None,
    names= cols,
    engine="python",
    encoding="latin1"
)

test_data = pd.read_csv(
    "/content/drive/My Drive/nlp/test.csv",
    header=None,
    names= cols,
    engine="python",
    encoding="latin1"
)

In [5]:
train_data.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
test_data.head()

,sentiment,id,date,query,user,text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...


In [7]:
train_data.shape

(1600000, 6)

In [8]:
test_data.shape

(498, 6)

In [0]:
# Preprocess

In [0]:
## cleaning
train_data.drop(["id","date","query", "user"],
          axis=1,
          inplace=True)

In [0]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, "lxml").get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r" +", " ", tweet)

  return tweet

In [0]:
data_clean = [clean_tweet(tweet) for tweet in train_data.text]

In [0]:
data_labels = train_data.sentiment.values
data_labels[data_labels == 4] = 1

In [14]:
set(data_labels)

{0, 1}

In [0]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size= 2**16
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [0]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

In [17]:
test_idx = np.random.randint(0, 800000, 8000)
print(len(test_idx))
test_idx = np.concatenate((test_idx, test_idx+800000))
print(len(test_idx))

8000
16000


In [18]:
len(test_idx)

16000

In [0]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)

In [0]:
## Buil the model

In [0]:
class DCNN(tf.keras.Model):
  def __init__(self,
               vocab_size,
               emb_dim=128,
               nb_filters=50,
               FFN_units=512,
               nb_classes=2,
               dropout_rate=0.1,
               training=False,
               name="dcnn"):
    super(DCNN, self).__init__(name=name)

    self.embedding = layers.Embedding(vocab_size,
                                      emb_dim)
    
    self.bigram = layers.Conv1D(filters=nb_filters,
                                kernel_size=2,
                                padding="valid",
                                activation="relu")
    self.pool_1 = layers.GlobalMaxPool1D()
    
    self.trigram = layers.Conv1D(filters=nb_filters,
                                kernel_size=3,
                                padding="valid",
                                activation="relu")
    self.pool_2 = layers.GlobalMaxPool1D()
    
    self.fourgram = layers.Conv1D(filters=nb_filters,
                                kernel_size=4,
                                padding="valid",
                                activation="relu")
    
    self.pool_3 = layers.GlobalMaxPool1D()

    self.dense_1 = layers.Dense(units=FFN_units, activation="relu")

    self.dropout = layers.Dropout(rate=dropout_rate)

    if nb_classes == 2:
      self.last_dense = layers.Dense(units=1,
                                     activation="sigmoid")
    else:
      self.last_dense = layers.Dense(units=nb_classes,
                                     activation="softmax")
      
  def call(self, inputs, training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool_1(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool_1(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool_1(x_3)

    merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training)
    output = self.last_dense(merged)

    return output






In [0]:
## Application


In [0]:
# config
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS= 256
NB_CLASSES = len(set(train_labels))

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 2

In [0]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [0]:
if NB_CLASSES == 2:
  Dcnn.compile(loss="binary_crossentropy",
               optimizer="adam",
               metrics=["accuracy"])
else:
  Dcnn.compile(loss="sparse_categorical_crossentropy",
               optimizer="adam",
               metrics=["sparse_categorical_accuracy"])

In [0]:
checkpoint_path = "./drive/My Drive/nlp/ckpt/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("Latest checkpoint restored!!")

In [31]:
Dcnn.fit(train_inputs,
         train_labels,
         batch_size = BATCH_SIZE,
         epochs = NB_EPOCHS)
ckpt_manager.save()

Train on 1584078 samples
Epoch 1/2
1584078/1584078 [==============================] - 3910s 2ms/sample - loss: 0.3988 - accuracy: 0.8202
Epoch 2/2
1584078/1584078 [==============================] - 3910s 2ms/sample - loss: 0.3335 - accuracy: 0.8569


'./drive/My Drive/nlp/ckpt/ckpt-1'

In [32]:
results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

16000/16000 [==============================] - 1s 70us/sample - loss: 0.3767 - accuracy: 0.8335
[0.37666514378786087, 0.8335]


In [33]:
Dcnn(np.array([tokenizer.encode("I love this!")]), training=False).numpy()

array([[0.7836227]], dtype=float32)

In [38]:
Dcnn(np.array([tokenizer.encode("The pages were torn and the book was damaged!")]), training=False).numpy()

array([[0.12052897]], dtype=float32)